In [ ]:
%matplotlib widget
"""Advent of Code - Day 22, Year 2023
Solution Started: Jan 11, 2025
Puzzle Link: https://adventofcode.com/2023/day/22
Solution by: abbasmoosajee07
Brief: [Tower of Bricks]
"""

#!/usr/bin/env python3

import os, re, copy, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MaxNLocator

start_time = time.time()

# Load the input data from the specified file path
D22_file = "Day22_input.txt"
D22_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), D22_file)

# Read and sort input data into a grid
with open(D22_file_path) as file:
    input_data = file.read().strip().split('\n')

def parse_input(input_list: list[str]) -> dict:
    graph = {}
    for brick_no, line in enumerate(input_list, start=1):
        # Parse the coordinates from the input string
        edge_1, edge_2 = line.split('~')
        edge_1_coords = tuple(map(int, edge_1.split(',')))
        edge_2_coords = tuple(map(int, edge_2.split(',')))
        # Store the brick name and the coordinates in the graph
        # print(brick_no, brick_name, edge_1_coords, edge_2_coords)
        graph[brick_no] = (edge_1_coords, edge_2_coords)
    return graph

def build_bricks(brick_edges: dict) -> dict:
    expanded_graph = []

    # Generate coordinates with their corresponding brick names
    for brick_name, (edge_1, edge_2) in brick_edges.items():
        # Unpack coordinates for readability and get min/max for each axis
        (x1, y1, z1), (x2, y2, z2) = edge_1, edge_2
        min_x, max_x = sorted([x1, x2])
        min_y, max_y = sorted([y1, y2])
        min_z, max_z = sorted([z1, z2])

        # Generate all coordinates for the brick
        expanded_graph.extend([(brick_name, (dx, dy, dz))
                                for dx in range(min_x, max_x + 1)
                                for dy in range(min_y, max_y + 1)
                                for dz in range(min_z, max_z + 1)])

    # Sort the graph by z-coordinate
    expanded_graph.sort(key=lambda x: x[1][2])

    # Rebuild the dictionary with sorted coordinates
    sorted_graph = {}
    for brick_name, coord in expanded_graph:
        sorted_graph.setdefault(brick_name, []).append(coord)

    return sorted_graph

def plot_tower(brick_coordinates: dict, title: str = '3D Scatter Plot with Connected Points'):
    """
    Plots 3D scatter and line plots for a given dictionary of brick coordinates.

    Args:
        brick_coordinates (dict): A dictionary where keys are brick names and values are lists of (x, y, z) coordinates.
        title (str): The title of the plot. Defaults to '3D Scatter Plot with Connected Points'.
    """
    # Create a 3D scatter plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Generate colors for unique groups
    colors = plt.cm.tab10(range(len(brick_coordinates)))

    # Plot each brick group with unique color and label
    for i, (name, points) in enumerate(brick_coordinates.items()):
        xs, ys, zs = zip(*points)  # Unpack x, y, z coordinates

        # Scatter plot for points
        ax.scatter(xs, ys, zs, label=name, color=colors[i])

        # Line plot to connect points
        ax.plot(xs, ys, zs, color=colors[i], alpha=1)

    # Set integer-only ticks
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.zaxis.set_major_locator(MaxNLocator(integer=True))

    # Set labels, title, and legend
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')
    ax.set_title(title)

    # Uncomment below to show a legend
    ax.legend(title='Groups')

    # Show the plot
    plt.show()

def build_tower(brick_graph: dict):
    blocks = list(brick_graph.values())
    N = len(blocks)
    blocks.sort(key=lambda x: x[1][2]) # sort by z1: distance to ground
    X_MAX = max(b[1][0] for b in blocks) + 1
    Y_MAX = max(b[1][1] for b in blocks) + 1
    Z_MAX = max(b[1][2] for b in blocks) + 1

    stack = [[['empty' for _ in range(X_MAX)]
                for _ in range(Y_MAX)]
                for _ in range(Z_MAX)]

    supported_by = {}

    for block_id, ((x1, y1, z1), (x2, y2, z2)) in enumerate(blocks):
        # Determine the height of the block
        height = z2 - z1 + 1

        # Let the block fall until it receives support
        for z in range(Z_MAX):
            # Collect support for the block at this level
            support = {
                stack[z][y][x]
                for x in range(x1, x2 + 1)
                for y in range(y1, y2 + 1)
            } - {'empty'}

            # If there is support, record it and stop falling
            if support:
                supported_by[block_id] = support
                break

        # Add the block above its support
        for z_ in range(z - height, z):
            for x in range(x1, x2 + 1):
                for y in range(y1, y2 + 1):
                    stack[z_][y][x] = block_id

    # Disintegrate indispensible blocks & examine the resulting chain reactions
    indispensible = set.union(*[x for x in supported_by.values() if len(x)==1])
    total = 0
    for i in indispensible:
        disintegrated = set([i])
        for j in range(i+1, N):
            if j in supported_by and supported_by[j].issubset(disintegrated):
                disintegrated.add(j)
        total += len(disintegrated) - 1

    return len(blocks) - len(indispensible), total

brick_graph = parse_input(input_data)
expanded_bricks = build_bricks(brick_graph)

ans_p1, ans_p2 = build_tower(brick_graph)
print("Part 1:", ans_p1)
print("Part 2:", ans_p2)

print(f"Execution Time = {time.time() - start_time:.5f}")

# Brick A is the only brick supporting bricks B and C.
# Brick B is one of two bricks supporting brick D and brick E.
# Brick C is the other brick supporting brick D and brick E.
# Brick D supports brick F.
# Brick E also supports brick F.
# Brick F supports brick G.
# Brick G isn't supporting any bricks.